# Developing Custom Image Classification Model

In this lab, you will evolve the custom image classification model developed in Lab1:

Developed | Cultivated | Barren
--------- | ------ | ----------
![Developed](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/developed1.png) | ![Cultivated](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/cultivated1.png) | ![Barren](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/barren1.png)

Forested | Grassland | Shrub
---------| ----------| -----
![Forested](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/forest1.png) | ![Grassland](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/grassland1.png) | ![Shrub](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/shrub1.png)

You will utilize a custom image classifier using a Deep Learning technique called *Fine-tuning*. *Fine-tuning* is a flavor of Transfer learning (demonstrated in Lab 1).

In fine-tuning, you remove the last layer(s) (usually the FCNN layers) of the pre-trained network and replace it with the new untrained layers that match the given ML task. You than re-train the full-network (pre-trained "trunk" and new top) using images from your custom domain. It is also a common practice to freeze the weights of the first few layers of the pre-trained network. This is because these layers capture universal features like curves and edges that are also relevant to the new problem. You want to keep those weights intact. Instead, you "force" the network to focus on learning dataset-specific features in the subsequent layers.

Since *fine-tuning* is much more computationally intensive than transfer learning approach used in Lab 1, we will train the network using the Horovod distributed training algorithm. Azure Machine Learning provides built-in support for Horovod that simpliefies cluster configuration and job scheduling.

![Fine-tuning](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/fine-tune.png)

## Download the lab dataset

Open the terminal window and download the image dataset to the `/tmp` directory.

```
cd /tmp
wget https://azureailabs.blob.core.windows.net/aerialtar/aerialsmall.tar.gz 
tar -xvf aerialsmall.tar.gz
```
    

In [6]:
%%sh

ls -l /tmp/aerialsmall

total 12
drwxr-xr-x 8 nbuser nbuser 4096 Oct  2 20:40 test
drwxr-xr-x 8 nbuser nbuser 4096 Oct  2 20:40 train
drwxr-xr-x 8 nbuser nbuser 4096 Oct  2 20:40 valid


## Connect to AML Workspace

In [7]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


In [8]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /home/nbuser/library/aml_config/config.json
jkamlws
jkamlws
southcentralus
952a710c-8d9c-40c1-9fec-f752138cc0b3


## Upload the dataset to the default Datastore

We will upload the dataset to the default Datastore to make it available to all nodes in the cluster.

In [9]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir='/tmp/aerialsmall', target_path='aerialsmall', overwrite=True, show_progress=True)

AzureBlob jkamlwsstorageijhxtdoi azureml-blobstore-68c45b05-3479-49da-8b04-2517f76acaf6
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_137749.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_51801.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_81541.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_50494.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_55337.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va025_2016_1_2006.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va025_2016_1_4847.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va025_2016_1_4908.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va053_2016_1_6206.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va135_2016_1_1485.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_va025_2016_1_3010.png
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca08

Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_114349.png, 31 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_66596.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca059_2016_1_23109.png, 32 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_69486.png, 33 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_106175.png, 34 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_67442.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_80809.png, 35 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_92523.png, 36 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_97256.png
Uploading /tmp/aerialsmall/train/Fores

Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_79917.png, 77 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_101367.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_43616.png, 78 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_102896.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_82117.png, 79 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_45537.png, 80 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_40186.png, 81 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_81001.png, 82 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca111_2016_1_45295.png, 83 files out of an estimated total of 

Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_100916.png, 119 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_80830.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_104306.png, 120 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_81690.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca083_2016_1_100121.png, 121 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca025_2016_1_38763.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_91933.png, 122 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_130739.png
Uploaded /tmp/aerialsmall/train/Forest/ortho_1-1_hn_s_ca037_2016_1_88035.png, 123 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca025_2016

Uploaded /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_137318.png, 167 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_48228.png
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_48239.png
Uploaded /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_45715.png, 168 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca025_2016_1_156685.png, 169 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_48240.png
Uploading /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_48322.png
Uploaded /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca025_2016_1_174041.png, 171 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/train/Developed/ortho_1-1_hn_s_ca037_2016_1_45103.png, 170 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/train/Developed/ortho

...

Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_211111.png, 4248 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_265336.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_195803.png, 4249 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_267247.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_215255.png, 4250 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_269073.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_197697.png, 4251 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_273037.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_191582.png, 4252 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/v

Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_123828.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_42201.png, 4291 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_39179.png, 4292 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_29269.png, 4293 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_126950.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_37344.png, 4294 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_130419.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_hn_s_ne031_2016_1_43998.png, 4295 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_130575.png
Uploaded /tmp/aerialsmall/valid/Herbaceous/ortho_1-1_

Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_92316.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_181315.png, 4341 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_95017.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_156583.png, 4342 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_170512.png, 4343 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_101882.png
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_107733.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_183553.png, 4344 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_109456.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca025_2016_1_183915.png, 4345 files out of an estimated total of 44

Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_159574.png, 4383 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca111_2016_1_51925.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_120351.png, 4384 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca111_2016_1_56572.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_98182.png, 4385 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca111_2016_1_57029.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_72963.png, 4386 files out of an estimated total of 4419
Uploading /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca111_2016_1_58047.png
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_63981.png, 4387 files out of an estimated total of 4419
Uploaded /tmp/aerialsmall/valid/Shrub/ortho_1-1_hn_s_ca037_2016_1_69843.png, 438

$AZUREML_DATAREFERENCE_6f34849f8c544dc3a9c0c5d94a73ee66

## Create Azure ML Managed Compute

To run the lab's scripts we will utilize Azure ML managed compute resources. Specifically, an autoscale cluster of *Standard_NC6* VMs (equipped with Tesla K80 GPU). 

In [10]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os


# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "Standard_NC6")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

found compute target. just use it. gpu-cluster


## Training

### Pre-training

Before you can retrain the weights in the base network it is recommend to pre-train the new top with all weights in the base network frozen. We will run pretraining for a few epochs on a single node of the cluster.

#### Create training script

In [11]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

In [12]:
%%writefile $script_folder/pre-train.py

import os
import numpy as np
import random
import h5py

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.applications import vgg16


from azureml.core import Run


# Create custom callback to track accuracy measures in AML Experiment
class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))


def custom_classifier(input_shape=(224,224,3), units=256, classes=6,  l1=0.01, l2=0.01, optimizer='adadelta'):
    # Create a base vgg16 model
    base_model = vgg16.VGG16(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False,
        pooling='avg')
    # Add new top
    x = base_model.output
    x = Dense(units, activation='relu')(x)
    x = Dropout(0.5)(x)
    y = Dense(classes, activation='softmax', kernel_regularizer=l1_l2(l1=l1, l2=l2))(x)
    model = Model(inputs=base_model.inputs, outputs=y)
    
    return model, base_model
       

def main(argv=None):
    
    
    print("Loading data from:", FLAGS.data_folder)
    # Create training and validation data generators
    train_data_dir = os.path.join(FLAGS.data_folder, 'train')
    valid_data_dir = os.path.join(FLAGS.data_folder, 'valid')
     
    # A hack to mitigate a bug in TF.Keras 1.12
    def preprocess_input_new(x):
        img = vgg16.preprocess_input(image.img_to_array(x))
        return image.array_to_img(img)
    
    batchsize=64
    classes = ["Barren", "Cultivated", "Developed", "Forest", "Herbaceous", "Shrub"]
    
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_new)
    train_generator = train_datagen.flow_from_directory(
        directory=train_data_dir,
        target_size=(224, 224),
        classes=classes,
        batch_size=batchsize)

    valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_new)
    valid_generator = train_datagen.flow_from_directory(
        directory=valid_data_dir,
        target_size=(224, 224),
        classes=classes,
        batch_size=batchsize)
    
    print(len(train_generator))
    print(len(valid_generator))
    
    
    # Create a custom model
    model, base_model = custom_classifier()
    
    # freeze all base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Use adadelta optimizer for pretraining the top layer
    model.compile(loss='categorical_crossentropy',
              optimizer = 'adadelta',
              metrics=['accuracy'])
    
    model.summary()
    
    
    # Configure callbacks to generate Tensorboard and AML logs
    run = Run.get_submitted_run()
    callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs'),
                 RunCallback(run)]
    
    # Start training
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=FLAGS.epochs,
        callbacks=callbacks,
        validation_data=valid_generator,
        validation_steps=len(valid_generator))
    
    # Save the trained model to outputs which is a standard folder expected by AML
    print("Training completed.")
    os.makedirs('outputs', exist_ok=True)
    model_file = os.path.join('outputs', 'aerial_model_pretrain.h5')
    weights_file = os.path.join('outputs', 'aerial_model_weights_pretrain.h5')
    print("Saving model to: {0}".format(model_file))
    model.save(model_file)
    print("Saving model weights to: {0}".format(weights_file))
    model.save_weights(weights_file)
 

# Default global parameters
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('batch_size', 32, "Number of images per batch")
tf.app.flags.DEFINE_integer('epochs', 10, "Number of epochs to train")
tf.app.flags.DEFINE_integer('units', 512, "Number of epochs to train")
tf.app.flags.DEFINE_string('data_folder', 'aerialsmall', "Folder with images")

if __name__ == '__main__':
    tf.app.run()
    

Writing ./script/pre-train.py


#### Create AML Experiment
We will track pre-traning in a dedicated Experiment.

In [13]:
from azureml.core import Experiment
experiment_name = 'aerial-finetune-pretrain'
exp = Experiment(workspace=ws, name=experiment_name)

#### Run a pre-training on a single node of the cluster

We will use *TensorFlow* estimator. *TensorFlow* estimator automatically configures the runtime image with all required pre-requisites - TensorFlow, CUDA, etc.

Due to time limitations of the lab, we will run pre-training for 3 epochs only. Note that in a real production scenario you would want to run pre-training for more epochs.

In [14]:
from azureml.train.dnn import TensorFlow

ds = ws.get_default_datastore()

script_params = {
    '--data_folder': ds.path('aerialsmall').as_mount(),
    '--epochs': 3
}

pip_packages = ['h5py', 'pillow', 'scipy']

est = TensorFlow(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='pre-train.py',
                use_gpu=True,
                pip_packages=pip_packages
                )


In [15]:
tags = {"Run Type": "Top pre-train"}
run = exp.submit(est, tags=tags)
run

Experiment,Id,Type,Status,Details Page,Docs Page
aerial-finetune-pretrain,aerial-finetune-pretrain_1549305084147,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


In [16]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

#### Monitor the run with Tensorboard

Azure Machine Learning has a built-in support for **Tensorboard**.

In [ ]:
from azureml.contrib.tensorboard import Tensorboard
tb = Tensorboard([run])
tb.start()

To connect to Tensorboard navigate to `http://<external IP of your DSVM>:6006`. If you have issues connecting, double check that you opened port 6006 as described in the lab set up instructions.

In [ ]:
tb.stop()

You can cancel the run with the `cancel` method.

In [ ]:
# run.cancel()

Or block till the run completes.

In [ ]:
#run.wait_for_completion(show_output=True)

#### Retrieve weights

The training scripts saves the weights of the pre-trained model into the `outputs` folder. This folder is automatically copied to the *Experiment* after the run completes.

In [17]:
print(run.get_file_names())

['azureml-logs/20_image_build_log.txt', 'azureml-logs/55_batchai_execution.txt', 'azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'azureml-logs/azureml.log', 'logs/events.out.tfevents.1549305722.26606c36dfb7422e94d1e7715b7faad1000000', 'outputs/aerial_model_pretrain.h5', 'outputs/aerial_model_weights_pretrain.h5']


In [18]:
run.download_file('outputs/aerial_model_weights_pretrain.h5', '/tmp/models/aerial_model_weights_pretrain.h5')

In [19]:
%%sh

ls /tmp/models

aerial_model_weights_pretrain.h5


#### Upload the weights to the default datastore

The fine tuning script (below) will download the pre-trained weights from the default datastore.

In [20]:
# Upload the dataset to the DataStore

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)
ds.upload(src_dir='/tmp/models', target_path='models', overwrite=True, show_progress=True)

AzureBlob jkamlwsstorageijhxtdoi azureml-blobstore-68c45b05-3479-49da-8b04-2517f76acaf6
Uploading /tmp/models/aerial_model_weights_pretrain.h5
Uploaded /tmp/models/aerial_model_weights_pretrain.h5, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_0dfa1bb578eb4c78be160abd618aa881

### Fine-tuning

We are now ready to run distributed training on AML Compute cluster.

#### Create training script

The training script uses Horovod API to coordinate distributed training. In addition to the top layers, we will also re-train the last convolutional layer in the base VGG16 network.

In [21]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

In [22]:
%%writefile $script_folder/fine-tune.py

import os
import numpy as np
import random
import h5py

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.applications import vgg16
from tensorflow.keras import optimizers

import horovod.tensorflow.keras as hvd

from azureml.core import Run

# Create custom callback to track accuracy measures in AML Experiment
class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))


        
def custom_classifier(input_shape=(224,224,3), units=256, classes=6,  l1=0.01, l2=0.01, optimizer='adadelta'):
    # Create a base vgg16 model
    base_model = vgg16.VGG16(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False,
        pooling='avg')
    # Add new top
    x = base_model.output
    x = Dense(units, activation='relu')(x)
    x = Dropout(0.5)(x)
    y = Dense(classes, activation='softmax', kernel_regularizer=l1_l2(l1=l1, l2=l2))(x)
    model = Model(inputs=base_model.inputs, outputs=y)
    
    return model, base_model
    

def main(argv=None):
    
    
    # Initialize Horovod
    hvd.init()
    
    # Horovod: pin GPU to be used to process local rank (one GPU per process)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    tf.keras.backend.set_session(tf.Session(config=config))

    print("Initialized Horovod")
    
    print("Loading data from:", FLAGS.data_folder)
    # Create training and validation data generators
    train_data_dir = os.path.join(FLAGS.data_folder, 'train')
    valid_data_dir = os.path.join(FLAGS.data_folder, 'valid')
    
    # A hack to mitigate a bug in TF.Keras 1.12
    def preprocess_input_new(x):
        img = vgg16.preprocess_input(image.img_to_array(x))
        return image.array_to_img(img)
    
    # Configure training and validation data generators
    batchsize=64
    classes = ["Barren", "Cultivated", "Developed", "Forest", "Herbaceous", "Shrub"]
    
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_new)
    train_generator = train_datagen.flow_from_directory(
        directory=train_data_dir,
        target_size=(224, 224),
        classes=classes,
        batch_size=batchsize)

    valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_new)
    valid_generator = train_datagen.flow_from_directory(
        directory=valid_data_dir,
        target_size=(224, 224),
        classes=classes,
        batch_size=batchsize)
    
    print(len(train_generator))
    print(len(valid_generator))
    
    
    # Create a custom model
    model, base_model = custom_classifier()
    
    # Load the weights pretrained in the previous step on the first worker, 
    # which will broadcast them to other workers.
    if hvd.rank() == 0:
        weights_file = os.path.join(FLAGS.weights_folder, FLAGS.weights_filename)
        model.load_weights(weights_file)
        print("------------------------------------")
        print("Loaded pre-trained weights on Rank 0")
    
    # Make last convolutional layer trainable
    for layer in base_model.layers[:14]:
        layer.trainable = False
    
    for layer in base_model.layers[14:]:
        layer.trainable = True

    # Wrap an optimizer in Horovod
    # For fine tuning use SGD with a low learning rate
    optimizer = hvd.DistributedOptimizer(optimizers.SGD(lr=1e-4, momentum=0.9))
    
    model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])
    
    # Configure callbacks
    callbacks = [
        # Horovod: broadcast initial variable states from rank 0 to all other processes.
        # This is necessary to ensure consistent initialization of all workers when
        # training is started with loaded weights.
        hvd.callbacks.BroadcastGlobalVariablesCallback(0),

        # Horovod: average metrics among workers at the end of every epoch.
        #
        # Note: This callback must be in the list before the ReduceLROnPlateau,
        # TensorBoard, or other metrics-based callbacks.
        hvd.callbacks.MetricAverageCallback()

    ]
            
    # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
    # Configure Tensorboard and Azure ML Tracking
    if hvd.rank() == 0:
        callbacks.append(tf.keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))
        callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='./logs'))
        run = Run.get_submitted_run()
        callbacks.append(RunCallback(run))
   
    model.summary()
    
    # Start trining
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator)//hvd.size(),
        epochs=FLAGS.epochs,
        validation_data=valid_generator,
        validation_steps=3*len(valid_generator)//hvd.size(),
        callbacks=callbacks)
    
    # Save the trained model to outputs folder on the first worker
    if hvd.rank() == 0:  
        print("Training completed.")
        os.makedirs('outputs', exist_ok=True)
        model_file = os.path.join('outputs', 'aerial_model_fine_tune.h5')
        model.save(model_file)


# Default global parameters
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('batch_size', 32, "Number of images per batch")
tf.app.flags.DEFINE_integer('epochs', 10, "Number of epochs to train")
tf.app.flags.DEFINE_integer('units', 512, "Number of epochs to train")
tf.app.flags.DEFINE_string('data_folder', 'aerialsmall', "Folder with images")
tf.app.flags.DEFINE_string('weights_folder', 'models', "Folder with model weights")
tf.app.flags.DEFINE_string('weights_filename', 'aerial_model_weights_pretrain.h5', "Folder with model weights")


if __name__ == '__main__':
 
    tf.app.run()
    

Writing ./script/fine-tune.py


#### Create AML Experiment
We will track the Horovod runs in a dedicated experiment.

In [23]:
from azureml.core import Experiment
experiment_name = 'aerial-finetune-train'
exp = Experiment(workspace=ws, name=experiment_name)

#### Run a fine-tuning training on  the cluster

*TensorFlow* estimator encapsulates idiosyncrasies of Horovod cluster configuration and job scheduling.

In [28]:
from azureml.train.estimator import Estimator

from azureml.train.dnn import TensorFlow

ds = ws.get_default_datastore()

script_params = {
    '--data_folder': ds.path('aerialsmall').as_mount(),
    '--weights_folder': ds.path('models').as_download(),
    '--epochs': 7
}

# We need to install the up to date version of Horovod
# since the version in a standard AML GPU image does not support
# horovod.tensorflow.keras
pip_packages = ['h5py', 'pillow', 'scipy', 'horovod']

est = TensorFlow(source_directory=script_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='fine-tune.py',
                      node_count=3,
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      use_gpu=True,
                      pip_packages=pip_packages)

In [29]:
tags = {"Run Type": "Top pre-train"}
run = exp.submit(est, tags=tags)
run

Experiment,Id,Type,Status,Details Page,Docs Page
aerial-finetune-train,aerial-finetune-train_1549308697301,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [30]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [27]:
run.cancel()